In [1]:
import websockets
import asyncio
import json
import logging

In [2]:
class JsonWebsocketConnection(object):

    def __init__(self,
                 websocket: websockets.WebSocketServerProtocol):
        
        logging.debug("incoming connection")
        self._websocket = websocket
        self._is_closed = False;
    
    def is_closed(self):
        return self.is_closed
    
    async def close(self):
        logging.debug("closing connection")
        try:
            if self._is_closed:
                return
            self._websocket.close()
            self._is_closed = True
        except Exception as e:
            logging.warning("error closing connection: %s", str(e))
    
    async def send(self, message: dict):
        string_message = json.dumps(message)
        logging.debug("sending message: %s", string_message)
        try:
            await self._websocket.send(string_message)
        except Exception as e:
            logging.warning("error sending message: %s", str(e))
    
    async def handle_message(self, message: dict):
        pass # override this function

    async def run(self):
        try:
            async for message in self._websocket:
                try:
                    json_message = json.loads(message)
                except ValueError as e:
                    logging.warning("received unprocessable message %s", str(e))
                await self.handle_message(message) 
        
        except Exception as e:
                logging.warning("error in websocket connection: %s", str(e))



class JsonWebsocketServer(object):
    def __init__(self, handler_class: JsonWebsocketConnection, host:str = 'localhost', port:int = 8765):
        self._host = host
        self._port = port
        self._handler_class = handler_class

    def run(self):
        async def main(websocket: websockets.WebSocketServerProtocol,
                       path: str):

            connection = self._handler_class(websocket)

            connection.run()
        
        start_server = websockets.serve(main, self._host, self._port)

        asyncio.get_event_loop().run_until_complete(start_server)
        asyncio.get_event_loop().run_forever()
        

def CrosswordConnection(JsonWebsocketConnection): 
    async def handle_message(self, message: dict):
        await self.send({"message": "hello"})
    



In [3]:
server = JsonWebsocketServer(CrosswordConnection)
server.run()

RuntimeError: This event loop is already running